In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import os, glob
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from src.preprocess import *
from src.utils import *
from src.loss import *

In [2]:
recon = [Reconstruction(date='2011_09_26'), Reconstruction(date='2011_09_28'),
         Reconstruction(date='2011_09_29'), Reconstruction(date='2011_09_30'),
         Reconstruction(date='2011_10_03')]

In [ ]:
#npl=np.transpose(images_l[idx].numpy(), (1, 2, 0))
#npr=np.transpose(images_r[idx].numpy(), (1, 2, 0))
#uni = np.dot(np.ones((npl.shape[0],npl.shape[1])), 100)
#loss, synthesized = recon[0].compute_loss(uni, npl, npr, 'L2R')
#print(loss)

In [ ]:
#plt.imshow(synthesized)
#plt.show()